In [27]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from datetime import datetime

Proposed Datasets
Show Info:
Season
Episode
Air date
Contestant 1
Contestant 2
Contestant 3
Returning Contestant

Show Details:
Episode
Round
Category
Clue
Value
Clue Order
Correct Response
Correct Responder
Incorrect Response 1
Incorrect Responder 1
Incorrect Response 2
Incorrect Responder 2
Incorrect Response 3
Incorrect Responder 3
Daily Double
Wager

Show Statistics:
J Round Score C1
J Round Score C2
J Round Score C3
DJ Round Score C1
DJ Round Score C2
DJ Round Score C3
FJ Wager C1
FJ Wager C2
FJ Wager C3
FJ Round Score C1
FJ Round Score C2
FJ Round Score C3


All Jeopardy! seasons:
http://www.j-archive.com/listseasons.php

## Season List

In [28]:
response = requests.get('http://www.j-archive.com/listseasons.php')

In [29]:
soup = BeautifulSoup(response.text)
#soup

In [30]:
season_urls = np.array(['http://www.j-archive.com/' + a['href'] for a in soup.findAll('a') 
                    if 'showseason' in a['href'] and 'http' not in a['href']])
#season_urls

In [6]:
seasons = pd.DataFrame(season_urls)
seasons.columns = ['season_url']
#seasons
seasons.to_csv('data/j_seasons.csv', index=False)

## Episode List

In [32]:
response = requests.get('http://www.j-archive.com/showseason.php?season=37')

In [34]:
soup = BeautifulSoup(response.text)
soup

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.1//EN" "http://www.w3.org/TR/xhtml11/DTD/xhtml11.dtd">
<html xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>J! Archive - Season 37</title>
<link href="http://www.j-archive.com/styles.css" rel="styleSheet" type="text/css"/>
<link href="http://www.j-archive.com/favicon.ico" rel="shortcut icon"/>
</head>
<body>
<div id="navbar">
<span id="navbarlogo"><a href="http://www.j-archive.com"><img alt="J! Archive" height="22" src="http://www.j-archive.com/j-a.gif" width="100"/></a></span>
<span id="navbartext"><a href="http://www.j-archive.com/showseason.php?season=37">[current season]</a> <a href="http://www.j-archive.com/showseason.php?season=36">[last season]</a> <a href="http://www.j-archive.com/listseasons.php">[all seasons]</a> <a href="http://www.j-archive.com/listprizes.php">[prizes]</a> <a href="http://www.j-archive.com/wageringcalculator.php">[wagering calculator]</a> <a href="http://www.j-archive.com/help.php">[help]</a></sp

In [43]:
season = soup.find('title').get_text()
#season = season.get_text()
#season
season = season.split('Season ')[1]
season

'37'

In [42]:
episodes = np.array([a['href'] for a in soup.findAll('a') if 'game_id' in a['href']])
episodes = pd.DataFrame(episodes)
episodes.columns = ['episode_url']
episodes['season'] = season
episodes

,episode_url,season
0,http://www.j-archive.com/showgame.php?game_id=...,37
1,http://www.j-archive.com/showgame.php?game_id=...,37
2,http://www.j-archive.com/showgame.php?game_id=...,37
3,http://www.j-archive.com/showgame.php?game_id=...,37
4,http://www.j-archive.com/showgame.php?game_id=...,37
...,...,...
73,http://www.j-archive.com/showgame.php?game_id=...,37
74,http://www.j-archive.com/showgame.php?game_id=...,37
75,http://www.j-archive.com/showgame.php?game_id=...,37
76,http://www.j-archive.com/showgame.php?game_id=...,37


In [10]:
episodes.to_csv('data/j_episodes.csv', index=False)

## Episode Details

In [44]:
#response = requests.get('http://www.j-archive.com/showgame.php?game_id=6895')
#response = requests.get('http://www.j-archive.com/showgame.php?game_id=6893')
response = requests.get('http://www.j-archive.com/showgame.php?game_id=6378')

In [45]:
soup = BeautifulSoup(response.text)
#soup

In [46]:
header = soup.find('h1')
header = header.text
show_num = header.split('-')[0].strip().split('#')[1]
air_date = datetime.strftime(datetime.strptime(header.split('-')[1].strip().split(',',1)[1].strip(),'%B %d, %Y'), '%Y-%m-%d')
print(show_num, ', ', air_date)

8039 ,  2019-07-18


In [47]:
season = soup.find('a', text='[current season]')
print(season)
season = season['href'].split('=')[1]

<a href="http://www.j-archive.com/showseason.php?season=37">[current season]</a>


In [93]:
scores = soup.findAll(['h3','table'])
score_texts = np.array([['Scores at the end of the Jeopardy! Round:', 'J'],
                        ['Scores at the end of the Double Jeopardy! Round:', 'DJ'],
                        ['Final scores:', 'FJ']])
j_round = []
j_name = []
j_score = []
for x in score_texts:
    for element in scores:
        if element.text == x[0]:
            while element.name != 'table':
                element = element.next_sibling
                #print('name: ',element.name)
            score_table = element.findAll('tr')
            [j_round.append(x[1]) for td in score_table[0].findAll('td')]
            [j_name.append(td.text) for td in score_table[0].findAll('td')]
            [j_score.append(int(re.sub('[$,]', '', td.text))) for td in score_table[1].findAll('td')]

j_scores = pd.DataFrame({'show_num': show_num, 'round_code': j_round, 'contestant': j_name, 'score': j_score})
j_scores

,show_num,round_code,contestant,score
0,8039,J,Roey,11200
1,8039,J,Amy,3600
2,8039,J,Nathan,4400
3,8039,DJ,Roey,19600
4,8039,DJ,Amy,7200
5,8039,DJ,Nathan,9800
6,8039,FJ,Roey,19600
7,8039,FJ,Amy,9900
8,8039,FJ,Nathan,19600


In [95]:

remarks_text = 'Coryat scores:'
j_right = []
j_ddright = []
j_wrong = []
j_ddwrong = []
for element in scores:
    if element.text == remarks_text:
        while element.name != 'table':
            element = element.next_sibling
        remarks_table = element.findAll('td', class_ = 'score_remarks')
        [j_right.append((re.findall('(\d{1,2}) R', x.text.split(',')[0]) or list('0'))[0]) for x in remarks_table]
        [j_ddright.append((re.findall('(\d{1}) DD', x.text.split(',')[0]) or list('0'))[0]) for x in remarks_table]
        [j_wrong.append((re.findall('(\d{1,2}) W', x.text.split(',')[1]) or list('0'))[0]) for x in remarks_table]
        [j_ddwrong.append((re.findall('(\d{1}) DD', x.text.split(',')[1]) or list('0'))[0]) for x in remarks_table]

j_remarks = pd.DataFrame({'contestant': j_name[:3], 'num_right': j_right, 'num_ddright': j_ddright, 'num_wrong': j_wrong,
                         'num_ddwrong': j_ddwrong})
j_remarks[['num_right', 'num_ddright', 'num_wrong', 'num_ddwrong']] = j_remarks[['num_right', 'num_ddright', 'num_wrong', 'num_ddwrong']].apply(pd.to_numeric)

In [96]:
j_scores = j_scores.merge(j_remarks, how = 'left', on='contestant')
j_scores

,show_num,round_code,contestant,score,num_right,num_ddright,num_wrong,num_ddwrong
0,8039,J,Roey,11200,27,1,4,0
1,8039,J,Amy,3600,13,0,2,0
2,8039,J,Nathan,4400,12,1,2,1
3,8039,DJ,Roey,19600,27,1,4,0
4,8039,DJ,Amy,7200,13,0,2,0
5,8039,DJ,Nathan,9800,12,1,2,1
6,8039,FJ,Roey,19600,27,1,4,0
7,8039,FJ,Amy,9900,13,0,2,0
8,8039,FJ,Nathan,19600,12,1,2,1


In [86]:
j_scores[j_scores.round_code=='FJ'].sort_values(['score'], ascending = False)

,show_num,round_code,contestant,score,num_right,num_ddright,num_wrong,num_ddwrong
6,8039,FJ,Roey,19600,27,1,4,0
8,8039,FJ,Nathan,19600,12,1,2,1
7,8039,FJ,Amy,9900,13,0,2,0


In [97]:
#finish_order = j_scores[['contestant','score']][j_scores.round_code =='FJ']
#finish_order = finish_order.sort_values(['score'], ascending = False)
#finish_order
finish_order = []
winner = j_scores[j_scores.round_code=='FJ'].sort_values(['score'], ascending = False)['contestant'].iloc[0]
second = j_scores[j_scores.round_code=='FJ'].sort_values(['score'], ascending = False)['contestant'].iloc[1]
third = j_scores[j_scores.round_code=='FJ'].sort_values(['score'], ascending = False)['contestant'].iloc[2]
finish_order.append([winner, 'Winner'])
finish_order.append([second, 'Second'])
finish_order.append([third, 'Third'])
finish_order = pd.DataFrame(finish_order)
finish_order.columns=(['contestant', 'finish'])

j_scores = j_scores.merge(finish_order, how = 'left', on = 'contestant')
#j_scores.to_csv('data/j_scores.csv', index=False, header=False)


In [98]:
j_scores

,show_num,round_code,contestant,score,num_right,num_ddright,num_wrong,num_ddwrong,finish
0,8039,J,Roey,11200,27,1,4,0,Winner
1,8039,J,Amy,3600,13,0,2,0,Third
2,8039,J,Nathan,4400,12,1,2,1,Second
3,8039,DJ,Roey,19600,27,1,4,0,Winner
4,8039,DJ,Amy,7200,13,0,2,0,Third
5,8039,DJ,Nathan,9800,12,1,2,1,Second
6,8039,FJ,Roey,19600,27,1,4,0,Winner
7,8039,FJ,Amy,9900,13,0,2,0,Third
8,8039,FJ,Nathan,19600,12,1,2,1,Second


In [15]:
categories = soup.findAll('td', attrs = {'class': 'category_name'})

In [16]:
categories = np.array([x.text for x in categories])
#categories

In [17]:
rounds = []
[[rounds.append(item) for x in range(6)] for item in ['J', 'DJ']]
rounds.append('FJ')
rounds = np.array(rounds)
#rounds

In [18]:
catnums = np.array(['1', '2', '3', '4', '5', '6', '1', '2', '3', '4', '5', '6', '1'])
#catnums

In [19]:
categories = pd.DataFrame({'category': categories, 'round_code': rounds, 'cat_num': catnums})

In [20]:
categories['season'] = season
categories['show_num'] = show_num
categories['air_date'] = air_date
#categories

In [21]:
#split the results by the "clue" class
clues = soup.find_all('td', attrs = {'class': 'clue'})
#print(type(clues))
#print(type(soup))

In [22]:
all_rows = []
for clue in clues:
    #print(type(clue))
    new_row = []
    #Get our clue number, which tells us the round, category, and board position
    clue_id = clue.find('td', class_='clue_text')
    if not clue_id == None:
        #print(clue_id)
        clue_id = clue.find('td', class_='clue_text')['id']
        #What round is this?
        round_code = clue_id.split('_')[1]
        cat_num = clue_id.split('_')[2] if not round_code == 'FJ' else 'checkme'
        clue_num = clue_id.split('_')[3] if not round_code == 'FJ' else 'checkme'
        if not round_code == 'FJ':
            #Get our clue value
            #DD does not have a clue_value, it has a clue_value_daily_double
            #For DD split out the dollar value and set the flag
            clue_value = clue.find('td', class_='clue_value_daily_double').text.split(' ')[1] if not clue.find('td', {'class':'clue_value'}) else clue.find('td', {'class':'clue_value'}).text
            daily_double = False if not clue.find('td', class_='clue_value_daily_double') else True
            clue_order = clue.find('td', class_='clue_order_number').text
            clue_text = clue.find('td', class_='clue_text').text
            corr_resp = BeautifulSoup(clue.find('div', onmouseover=True).get('onmouseover'))
            #print(corr_resp)
            #Check for clues with no correct responder, and set the flag
            corr_respr1 = '' if not corr_resp.find('td', class_='right') else corr_resp.find('td', class_='right').text
            corr_FJ_wager1 = ''
            corr_respr2 = ''
            corr_FJ_wager2 = ''
            corr_respr3 = ''
            corr_FJ_wager3 = ''
            triple_stumper = True if not corr_resp.find('td', class_='right') else False
            incorr_resprs = corr_resp.findAll('td', class_='wrong')
            #Okay, THIS is ugly
            incorr_respr1 = ''
            incorr_FJ_wager1 = ''
            incorr_respr2 = ''
            incorr_FJ_wager2 = ''
            incorr_respr3 = ''
            incorr_FJ_wager3 = ''
            if len(incorr_resprs) >= 1:
                incorr_respr1 = '' if incorr_resprs[0].text == 'Triple Stumper' else incorr_resprs[0].text
            if len(incorr_resprs) >= 2:
                incorr_respr2 = '' if incorr_resprs[1].text == 'Triple Stumper' else incorr_resprs[1].text
            if len(incorr_resprs) == 3:
                incorr_respr3 = '' if incorr_resprs[2].text == 'Triple Stumper' else incorr_resprs[2].text
            corr_resp = corr_resp.find('em', class_='correct_response').text
            new_row = [round_code, cat_num, clue_num, clue_id, clue_value, daily_double, clue_order, clue_text, corr_resp, 
                       corr_respr1, corr_FJ_wager1, corr_respr2, corr_FJ_wager2, corr_respr3, corr_FJ_wager3, 
                       triple_stumper, incorr_respr1, incorr_FJ_wager1, incorr_respr2, incorr_FJ_wager2, incorr_respr3, incorr_FJ_wager3 ]
            all_rows.append(new_row)


In [23]:
all_rows

[['J',
  '1',
  '1',
  'clue_J_1_1',
  '$100',
  False,
  '25',
  'While in exile on the island of St. Helena from 1815-1821, he dictated his memoirs to friends',
  'Napoleon',
  'Mandi',
  '',
  '',
  '',
  '',
  '',
  False,
  '',
  '',
  '',
  '',
  '',
  ''],
 ['J',
  '2',
  '1',
  'clue_J_2_1',
  '$100',
  False,
  '16',
  "He scored 12 career goals in World Cup competition; now he's Brazil's special minister for sports",
  'Pele',
  'Mandi',
  '',
  '',
  '',
  '',
  '',
  False,
  '',
  '',
  '',
  '',
  '',
  ''],
 ['J',
  '3',
  '1',
  'clue_J_3_1',
  '$100',
  False,
  '6',
  'Waikiki is probably the most famous beach in this U.S. state',
  'Hawaii',
  'Soren',
  '',
  '',
  '',
  '',
  '',
  False,
  '',
  '',
  '',
  '',
  '',
  ''],
 ['J',
  '4',
  '1',
  'clue_J_4_1',
  '$100',
  False,
  '1',
  'The phrase "fugit ireparabile tempus" has been shortened to just "tempus fugit" - this "flies"',
  'Time',
  'Tom',
  '',
  '',
  '',
  '',
  '',
  False,
  '',
  '',
  '',
  '',

In [24]:
round_code = 'FJ'
cat_num = '1'
clue_num = '1'
clue_id = 'clue_FJ'
clue_value = ''
daily_double = False
clue_order = ''
clue_text = ''
corr_resp = ''
triple_stumper = True

final_round = soup.find('table', class_ = 'final_round').find_all('td')
#type(final_round)
#final_round - this will absolutely barf and so will a print; it gives a type error

for item in final_round:
    #this only works because it's the first item! aaargh
    stats = BeautifulSoup(item.find('div', onmouseover=True).get('onmouseover'))
    if stats:
        break
corr_resp = stats.find('em')
corr_resp = corr_resp.get_text()

#make a dictionary of responders
resprs = {}
wagers = stats.findAll('td')

right = 0
wrong = 0
for td in wagers:
    td_text = td.get_text()
    if td.has_attr('class'):
        td_class = td.get('class')[0]
        if td_class == 'right':
            triple_stumper = False
            right +=1
            key_name = 'corr_respr' + str(right)
            resprs[key_name] = td_text
            wager_key = 'corr_FJ_wager' + str(right)
        else:
            wrong +=1
            key_name = 'incorr_respr' + str(wrong)
            resprs[key_name] = td_text
            wager_key = 'incorr_FJ_wager' + str(wrong)
    if td.has_attr('rowspan'):
        continue
    if not td.has_attr('class'):
        resprs[wager_key] = td_text
        
corr_respr1 = resprs.get('corr_respr1', '')
corr_FJ_wager1 = resprs.get('corr_FJ_wager1', '')
corr_respr2 = resprs.get('corr_respr2', '')
corr_FJ_wager2 = resprs.get('corr_FJ_wager2', '')
corr_respr3 = resprs.get('corr_respr3', '')
corr_FJ_wager3 = resprs.get('corr_FJ_wager3', '')
incorr_respr1 = resprs.get('incorr_respr1','')
incorr_FJ_wager1 = resprs.get('incorr_FJ_wager1', '')
incorr_respr2 = resprs.get('incorr_respr2','')
incorr_FJ_wager2 = resprs.get('incorr_FJ_wager2', '')
incorr_respr3 = resprs.get('incorr_respr3','')
incorr_FJ_wager3 = resprs.get('incorr_FJ_wager3', '')

clue_text = soup.find('td', attrs = {'class': 'clue_text', 'id': 'clue_FJ'})
clue_text = clue_text.get_text()
#print(clue_text)

new_row = [round_code, cat_num, clue_num, clue_id, clue_value, daily_double, clue_order, clue_text, corr_resp, 
                   corr_respr1, corr_FJ_wager1, corr_respr2, corr_FJ_wager2, corr_respr3, corr_FJ_wager3, 
                   triple_stumper, incorr_respr1, incorr_FJ_wager1, incorr_respr2, incorr_FJ_wager2, incorr_respr3, incorr_FJ_wager3]
all_rows.append(new_row)

In [25]:
this_show = pd.DataFrame(all_rows)
this_show.columns = ['round_code', 'cat_num', 'clue_num', 'clue_id', 'clue_value', 'daily_double', 'clue_order', 'clue_text', 'corr_resp', 
                   'corr_respr1', 'corr_FJ_wager1', 'corr_respr2', 'corr_FJ_wager2', 'corr_respr3', 'corr_FJ_wager3', 
                   'triple_stumper', 'incorr_respr1', 'incorr_FJ_wager1', 'incorr_respr2', 'incorr_FJ_wager2', 'incorr_respr3', 'incorr_FJ_wager3']
this_show = this_show.merge(categories, how='left', left_on=['round_code', 'cat_num'], right_on=['round_code', 'cat_num'])
this_show

,round_code,cat_num,clue_num,clue_id,clue_value,daily_double,clue_order,clue_text,corr_resp,corr_respr1,...,incorr_respr1,incorr_FJ_wager1,incorr_respr2,incorr_FJ_wager2,incorr_respr3,incorr_FJ_wager3,category,season,show_num,air_date
0,J,1,1,clue_J_1_1,$100,False,25,While in exile on the island of St. Helena fro...,Napoleon,Mandi,...,,,,,,,EUROPEAN HISTORY,37,2752,1996-07-16
1,J,2,1,clue_J_2_1,$100,False,16,He scored 12 career goals in World Cup competi...,Pele,Mandi,...,,,,,,,SPORTS,37,2752,1996-07-16
2,J,3,1,clue_J_3_1,$100,False,6,Waikiki is probably the most famous beach in t...,Hawaii,Soren,...,,,,,,,TRAVEL U.S.A.,37,2752,1996-07-16
3,J,4,1,clue_J_4_1,$100,False,1,"The phrase ""fugit ireparabile tempus"" has been...",Time,Tom,...,,,,,,,LATIN WORDS & PHRASES,37,2752,1996-07-16
4,J,5,1,clue_J_5_1,$100,False,11,"Broken Hill, this country's largest company, t...",Australia,Tom,...,,,,,,,BUSINESS & INDUSTRY,37,2752,1996-07-16
5,J,6,1,clue_J_6_1,$100,False,26,Leo I asserted that papal power was granted by...,St. Peter,Tom,...,,,,,,,POPES NAMED LEO,37,2752,1996-07-16
6,J,1,2,clue_J_1_2,$200,False,21,This grand duchy was founded in 963 when Siegf...,Luxembourg,Mandi,...,,,,,,,EUROPEAN HISTORY,37,2752,1996-07-16
7,J,2,2,clue_J_2_2,$200,False,17,"From 1987-1995, this German tennis star won 18...",Steffi Graf,Mandi,...,,,,,,,SPORTS,37,2752,1996-07-16
8,J,3,2,clue_J_3_2,$200,False,7,Fans of Davy Crockett remember the Alamo is in...,Texas,Tom,...,,,,,,,TRAVEL U.S.A.,37,2752,1996-07-16
9,J,4,2,clue_J_4_2,$200,False,2,"Latin title of the Christmas hymn ""O Come All ...","""Adeste Fideles""",Mandi,...,,,,,,,LATIN WORDS & PHRASES,37,2752,1996-07-16


In [26]:
this_show.to_csv('data/j_clues01.csv', index=False)

In [ ]:
#this_show.to_csv('data/j_clues.csv', mode='a', index=False, header=False)

https://github.com/whymarrh/jeopardy-parser/blob/master/parser.py